In [1]:
polling_period_in_seconds = 1
monitoring_duration_in_seconds_this_notebook=1
monitoring_duration_in_seconds_other_notebooks=600

token = '?token=1e7a29fa159723bc8a734a7516c7fd826149d240dcc23731'

In [2]:
!hostname

MS.local


In [3]:
!jupyter --version

jupyter core     : 4.5.0
jupyter-notebook : 6.0.1
qtconsole        : 4.5.5
ipython          : 7.8.0
ipykernel        : 5.1.2
jupyter client   : 5.3.3
jupyter lab      : 1.1.4
nbconvert        : 5.6.0
ipywidgets       : 7.5.1
nbformat         : 4.4.0
traitlets        : 4.3.3


https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

potential additional work:
* add strace capture
 * https://github.com/blapid/strace2py
 * https://python-ptrace.readthedocs.io/en/latest/syscall.html
 * https://github.com/dirtyharrycallahan/pystrace

In [4]:
!pip install psutil

You should consider upgrading via the '/Users/msarica/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
print('pandas',pandas.__version__)
import matplotlib.pyplot as plt

pandas 0.25.1


# one-time system details

In [6]:
print(psutil.cpu_freq())

scpufreq(current=2300, min=2300, max=2300)


In [7]:
pid = os.getpid()
print(pid)

44893


In [8]:
py = psutil.Process(pid)
print(py)

psutil.Process(pid=44893, name='python3.7', started='22:20:46')


In [9]:
py.name()

'python3.7'

In [10]:
py.exe()

'/Users/msarica/anaconda3/bin/python3.7'

In [11]:
py.cwd()

'/Users/msarica/Desktop/UMBC/DS601_Spring21/Week9/notebook/host_streaming_stats'

In [12]:
py.cmdline()

['/Users/msarica/anaconda3/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/Users/msarica/Library/Jupyter/runtime/kernel-3e559dea-d5d5-40ec-8578-c2fd17042ba0.json']

# system performance

In [13]:
# gives a single float value
psutil.cpu_percent(percpu=True)

[55.1,
 1.1,
 30.3,
 1.1,
 21.1,
 0.0,
 12.2,
 0.0,
 9.0,
 0.0,
 23.6,
 0.0,
 10.1,
 0.0,
 3.3,
 0.0]

In [14]:
psutil.cpu_times_percent(percpu=True)

[scputimes(user=35.0, nice=0.0, system=15.0, idle=40.0),
 scputimes(user=1.0, nice=0.0, system=0.0, idle=90.0),
 scputimes(user=20.0, nice=0.0, system=7.0, idle=63.0),
 scputimes(user=0.0, nice=0.0, system=1.0, idle=90.0),
 scputimes(user=15.0, nice=0.0, system=5.0, idle=72.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=91.0),
 scputimes(user=8.0, nice=0.0, system=3.0, idle=79.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=91.0),
 scputimes(user=8.0, nice=0.0, system=1.0, idle=81.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=90.0),
 scputimes(user=19.0, nice=0.0, system=2.0, idle=69.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=91.0),
 scputimes(user=8.0, nice=0.0, system=1.0, idle=80.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=90.0),
 scputimes(user=2.0, nice=0.0, system=1.0, idle=89.0),
 scputimes(user=0.0, nice=0.0, system=0.0, idle=90.0)]

In [15]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

{'user': 1.0, 'nice': 0.0, 'system': 1.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 1.0, 'nice': 0.0, 'system': 1.0, 'idle': 0.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 0.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 1.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 1.0, 'idle': 1.0}
{'user': 1.0, 'nice': 0.0, 'system': 1.0, 'idle': 0.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 2.0}
{'user': 1.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 2.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 1.0}
{'user': 0.0, 'nice': 0.0, 'system': 0.0, 'idle': 2.0}


In [16]:
# gives an object with many fields
psutil.virtual_memory()

svmem(total=17179869184, available=6375161856, percent=62.9, used=9769431040, free=15867904, active=6365327360, inactive=6059552768, wired=3404103680)

In [17]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 17179869184,
 'available': 6374674432,
 'percent': 62.9,
 'used': 9769738240,
 'free': 15380480,
 'active': 6365364224,
 'inactive': 6059552768,
 'wired': 3404374016}

In [18]:
dict(psutil.swap_memory()._asdict())

{'total': 2147483648,
 'used': 792723456,
 'free': 1354760192,
 'percent': 36.9,
 'sin': 200032866304,
 'sout': 508891136}

In [19]:
dict(psutil.cpu_stats()._asdict())

{'ctx_switches': 76593,
 'interrupts': 831702,
 'soft_interrupts': 4234913508,
 'syscalls': 1479878}

In [20]:
psutil.disk_io_counters(perdisk=True).keys()

dict_keys(['disk0'])

In [21]:
dict(psutil.disk_io_counters(perdisk=True)['disk0']._asdict())

{'read_count': 18943340,
 'write_count': 13483543,
 'read_bytes': 411029061632,
 'write_bytes': 252528156672,
 'read_time': 11914980,
 'write_time': 11015836}

In [22]:
dir(psutil)

['AF_LINK',
 'AIX',
 'AccessDenied',
 'BSD',
 'CONN_CLOSE',
 'CONN_CLOSE_WAIT',
 'CONN_CLOSING',
 'CONN_ESTABLISHED',
 'CONN_FIN_WAIT1',
 'CONN_FIN_WAIT2',
 'CONN_LAST_ACK',
 'CONN_LISTEN',
 'CONN_NONE',
 'CONN_SYN_RECV',
 'CONN_SYN_SENT',
 'CONN_TIME_WAIT',
 'Error',
 'FREEBSD',
 'LINUX',
 'MACOS',
 'NETBSD',
 'NIC_DUPLEX_FULL',
 'NIC_DUPLEX_HALF',
 'NIC_DUPLEX_UNKNOWN',
 'NoSuchProcess',
 'OPENBSD',
 'OSX',
 'POSIX',
 'POWER_TIME_UNKNOWN',
 'POWER_TIME_UNLIMITED',
 'Popen',
 'Process',
 'STATUS_DEAD',
 'STATUS_DISK_SLEEP',
 'STATUS_IDLE',
 'STATUS_LOCKED',
 'STATUS_PARKED',
 'STATUS_RUNNING',
 'STATUS_SLEEPING',
 'STATUS_STOPPED',
 'STATUS_TRACING_STOP',
 'STATUS_WAITING',
 'STATUS_WAKING',
 'STATUS_ZOMBIE',
 'SUNOS',
 'TimeoutExpired',
 'WINDOWS',
 'ZombieProcess',
 '_LOWEST_PID',
 '_PY3',
 '_TOTAL_PHYMEM',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_as_di

In [23]:
# psutil.sensors_temperatures()  # linux only

In [24]:
psutil.sensors_battery()

sbattery(percent=37, secsleft=8760, power_plugged=False)

### put everything into a single row

In [25]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input, got",type(row))
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)[disk_label]._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [26]:
os.getpid()

44893

In [27]:
type(os.getpid())

int

In [28]:
py = psutil.Process(os.getpid())
print(py)

psutil.Process(pid=44893, name='python3.7', started='22:20:46')


In [29]:
# dir(py)

In [30]:
dict(py.memory_info()._asdict())

{'rss': 113725440, 'vms': 4804452352, 'pfaults': 45693, 'pageins': 280}

In [31]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

memory use: 0.10591506958007812 GB


In [32]:
dict(py.memory_full_info()._asdict())

{'rss': 113725440,
 'vms': 4804452352,
 'pfaults': 45694,
 'pageins': 280,
 'uss': 94605312}

In [33]:
py.cpu_percent()

0.0

In [34]:
dict(py.cpu_times()._asdict())

{'user': 1.39381312,
 'system': 0.470424896,
 'children_user': 0.0,
 'children_system': 0.0}

In [35]:
py.cmdline()

['/Users/msarica/anaconda3/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/Users/msarica/Library/Jupyter/runtime/kernel-3e559dea-d5d5-40ec-8578-c2fd17042ba0.json']

In [36]:
dict(py.num_ctx_switches()._asdict())

{'voluntary': 140728898439810, 'involuntary': 0}

In [37]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input for row, got",type(row))
        return None
    if not isinstance(pid,int):
        raise TypeError("ERROR: expected a int as input for pid, got",type(pid))
        return None
    
    #if psutil.Process(pid).is_running():
    for p in psutil.process_iter():
        if (pid==p.pid):
            try:
                mem_dict=dict(psutil.Process(pid).memory_full_info()._asdict())
            except psutil.NoSuchProcess:
                return row
            for k,v in mem_dict.items():
                row['process memory: '+k]=v

    try:
        cpu_times_dict=dict(psutil.Process(pid).cpu_times()._asdict())
    except psutil.NoSuchProcess:
        return row
    for k,v in cpu_times_dict.items():
        row['process cpu times: '+k]=v

    try:
        context_switches_dict=dict(psutil.Process(pid).num_ctx_switches()._asdict())
    except psutil.NoSuchProcess:
        return row        
    for k,v in context_switches_dict.items():
        row['process number context switches: '+k]=v

#     try:
#         io_counter_dict = dict(psutil.Process(pid).io_counters()._asdict())
#     except psutil.NoSuchProcess:
#         return row
#     for k,v in io_counter_dict.items():
#         row['process io counters: '+k]=v
    return row

# gather data and create plots for this notebook

In [61]:
list_of_rows=[]
start_time = time.time()
for indx in range(monitoring_duration_in_seconds_this_notebook):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

elapsed time: 1.0203118324279785 seconds


In [66]:
print(df.shape)
df.head()

list(df.columns)

(1, 117)


['CPU percent for 0',
 'CPU percent for 1',
 'CPU percent for 2',
 'CPU percent for 3',
 'CPU percent for 4',
 'CPU percent for 5',
 'CPU percent for 6',
 'CPU percent for 7',
 'CPU percent for 8',
 'CPU percent for 9',
 'CPU percent for 10',
 'CPU percent for 11',
 'CPU percent for 12',
 'CPU percent for 13',
 'CPU percent for 14',
 'CPU percent for 15',
 'time in unix seconds',
 'datetime',
 'CPU 1 times user',
 'CPU 1 times nice',
 'CPU 1 times system',
 'CPU 1 times idle',
 'CPU 2 times user',
 'CPU 2 times nice',
 'CPU 2 times system',
 'CPU 2 times idle',
 'CPU 3 times user',
 'CPU 3 times nice',
 'CPU 3 times system',
 'CPU 3 times idle',
 'CPU 4 times user',
 'CPU 4 times nice',
 'CPU 4 times system',
 'CPU 4 times idle',
 'CPU 5 times user',
 'CPU 5 times nice',
 'CPU 5 times system',
 'CPU 5 times idle',
 'CPU 6 times user',
 'CPU 6 times nice',
 'CPU 6 times system',
 'CPU 6 times idle',
 'CPU 7 times user',
 'CPU 7 times nice',
 'CPU 7 times system',
 'CPU 7 times idle',
 '

In [40]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [41]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        raise TypeError('ERROR: expected input of type dataframe, got',type(df))
        return None
    if not isinstance(col_name,str):
        raise TypeError('ERROR: expected input of type str, got',type(col_name))
        return None
    plt.plot_date(x=df['datetime'],y=df[col_name])
    plt.xticks(rotation=40)
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return

In [42]:
# for col_name in df.columns:
#     if 'seconds' not in col_name:
# #        plt.scatter(x=df['time in unix seconds'],y=df[col_name])
#         scatter_plot_column(df,col_name)
#     else:
#         print(col_name)

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

works even when the kernel of the other notebook is restarted

detects when new newbook kernels being added

https://github.com/iiSeymour/nbtop/blob/master/nbtop/main.py

### get a list of all active sessions

In [43]:
r = requests.get('http://localhost:8888/api/sessions' + token)
r.status_code
# http://localhost:8888/?token=1c7736ce797a8e7f87082b97d954e2c4402d4eb8fd990902

200

In [44]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

{'id': '14c84438-8bf5-48ab-8f8d-970b045266d7',
 'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
 'name': '',
 'type': 'notebook',
 'kernel': {'id': 'f8316b7c-7ceb-479a-a5bc-d4a53654ba41',
  'name': 'python3',
  'last_activity': '2021-03-24T02:20:35.329250Z',
  'execution_state': 'idle',
  'connections': 1},
 'notebook': {'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
  'name': ''}}

### from json returned from API, we can extract the notebook kernel IDs

In [45]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

notebook kernel IDs:
f8316b7c-7ceb-479a-a5bc-d4a53654ba41
5af562e1-ad29-4f51-997e-4aedc17c9212
3e559dea-d5d5-40ec-8578-c2fd17042ba0


### get a list of all pids
 https://github.com/giampaolo/psutil/issues/132

In [46]:
print('all processes running:')
for p in psutil.process_iter():
    try:
        print(p)
    except psutil.NoSuchProcess:
        pass

all processes running:
psutil.Process(pid=0, name='kernel_task', started='2021-03-04 02:13:09')
psutil.Process(pid=1, name='launchd', started='2021-03-04 02:13:09')
psutil.Process(pid=55, name='syslogd', started='2021-03-04 02:13:22')
psutil.Process(pid=56, name='UserEventAgent', started='2021-03-04 02:13:22')
psutil.Process(pid=59, name='uninstalld', started='2021-03-04 02:13:22')
psutil.Process(pid=60, name='fseventsd', started='2021-03-04 02:13:22')
psutil.Process(pid=61, name='mediaremoted', started='2021-03-04 02:13:22')
psutil.Process(pid=64, name='systemstats', started='2021-03-04 02:13:22')
psutil.Process(pid=66, name='configd', started='2021-03-04 02:13:22')
psutil.Process(pid=67, name='endpointsecurityd', started='2021-03-04 02:13:22')
psutil.Process(pid=68, name='powerd', started='2021-03-04 02:13:22')
psutil.Process(pid=71, name='remoted', started='2021-03-04 02:13:22')
psutil.Process(pid=73, name='logd', started='2021-03-04 02:13:22')
psutil.Process(pid=74, name='keybagd',

### get pid for each notebook

In [47]:
def get_all_kernel_pids():
    dict_of_jupyter_kernel_pids = {}
    for p in psutil.process_iter():
        try:
            list_of_commands_per_process = psutil.Process(p.pid).cmdline()
    #        print(list_of_commands_per_process)
            if ('ipykernel_launcher' in list_of_commands_per_process):
                dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        except:
            pass
    return dict_of_jupyter_kernel_pids

get_all_kernel_pids()

{44880: '/Users/msarica/Library/Jupyter/runtime/kernel-f8316b7c-7ceb-479a-a5bc-d4a53654ba41.json',
 44885: '/Users/msarica/Library/Jupyter/runtime/kernel-5af562e1-ad29-4f51-997e-4aedc17c9212.json',
 44893: '/Users/msarica/Library/Jupyter/runtime/kernel-3e559dea-d5d5-40ec-8578-c2fd17042ba0.json'}

In [48]:
dict_of_jupyter_kernel_pids = get_all_kernel_pids()
dict_of_jupyter_kernel_pids

{44880: '/Users/msarica/Library/Jupyter/runtime/kernel-f8316b7c-7ceb-479a-a5bc-d4a53654ba41.json',
 44885: '/Users/msarica/Library/Jupyter/runtime/kernel-5af562e1-ad29-4f51-997e-4aedc17c9212.json',
 44893: '/Users/msarica/Library/Jupyter/runtime/kernel-3e559dea-d5d5-40ec-8578-c2fd17042ba0.json'}

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [49]:
def add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts):
    if not isinstance(dict_of_jupyter_kernel_pids,dict):
        raise TypeError("ERROR: expected a dict as input for dict_of_jupyter_kernel_pids, got",type(dict_of_jupyter_kernel_pids))
        return None
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
        for notebook_indx in range(len(list_of_kernel_dicts)):
            if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#                print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
                list_of_kernel_dicts[notebook_indx]['pid']=pid
    for kernel_dict in list_of_kernel_dicts:
        if 'pid' not in kernel_dict.keys():
            print('this kernel_dict is missing a pid:')
            print(kernel_dict)
            print('and the list of PIDs is')
            print(dict_of_jupyter_kernel_pids)
    return list_of_kernel_dicts

In [50]:
list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)

In [51]:
list_of_kernel_dicts

[{'id': '14c84438-8bf5-48ab-8f8d-970b045266d7',
  'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
  'name': '',
  'type': 'notebook',
  'kernel': {'id': 'f8316b7c-7ceb-479a-a5bc-d4a53654ba41',
   'name': 'python3',
   'last_activity': '2021-03-24T02:20:35.329250Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
   'name': ''},
  'pid': 44880},
 {'id': '5a8b616f-6514-4957-9416-8e22f4da9ea2',
  'path': 'UMBC/DS601_Spring21/Week9/notebook/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': '',
  'type': 'notebook',
  'kernel': {'id': '5af562e1-ad29-4f51-997e-4aedc17c9212',
   'name': 'python3',
   'last_activity': '2021-03-24T02:20:39.192852Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'UMBC/DS601_Spring21/Week9/notebook/host_streaming_stats/analysis_of_static_timeseries.ipynb',
   'name': ''},
  'pid': 44885},


### which kernels are not this notebook?

In [52]:
def kernel_dicts_other_notebooks(list_of_kernel_dicts):
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    list_of_kernel_dicts_for_other_notebooks=[]
    for kernel_dict in list_of_kernel_dicts:
        if 'pid' in kernel_dict.keys():
            if (str(os.getpid())!=str(kernel_dict['pid'])):
                list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)
        else:
            print('pid not found in key for kernel_dicts')
            print(list_of_kernel_dicts)
    return list_of_kernel_dicts_for_other_notebooks

In [53]:
list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)
list_of_kernel_dicts_for_other_notebooks

[{'id': '14c84438-8bf5-48ab-8f8d-970b045266d7',
  'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
  'name': '',
  'type': 'notebook',
  'kernel': {'id': 'f8316b7c-7ceb-479a-a5bc-d4a53654ba41',
   'name': 'python3',
   'last_activity': '2021-03-24T02:20:35.329250Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'UMBC/DS601_Spring21/Week9/notebook/4_plotting_timeseries_data.ipynb',
   'name': ''},
  'pid': 44880},
 {'id': '5a8b616f-6514-4957-9416-8e22f4da9ea2',
  'path': 'UMBC/DS601_Spring21/Week9/notebook/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': '',
  'type': 'notebook',
  'kernel': {'id': '5af562e1-ad29-4f51-997e-4aedc17c9212',
   'name': 'python3',
   'last_activity': '2021-03-24T02:20:39.192852Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'UMBC/DS601_Spring21/Week9/notebook/host_streaming_stats/analysis_of_static_timeseries.ipynb',
   'name': ''},
  'pid': 44885}]

In [54]:
def get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks,
                          url_for_api_sessions='http://localhost:8888/api/sessions' + token):
    if not isinstance(list_of_kernel_dicts_for_other_notebooks,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts_for_other_notebooks, got",type(list_of_kernel_dicts_for_other_notebooks))
        return None

    list_of_kernel_dicts = requests.get(url_for_api_sessions).json()
    dict_of_jupyter_kernel_pids = get_all_kernel_pids()
    list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)
    current_list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)

    list_of_pids_in_previous=[]
    for old_dict in list_of_kernel_dicts_for_other_notebooks:
        list_of_pids_in_previous.append(old_dict['pid'])
    for curr_dict in current_list_of_kernel_dicts_for_other_notebooks:
        if (curr_dict['pid'] not in list_of_pids_in_previous):
            list_of_kernel_dicts_for_other_notebooks.append(curr_dict)
    return list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [55]:
#monitoring_duration_in_seconds_other_notebooks=60 # 60 queries takes 63 seconds

In [57]:
nb_dict={}
list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids([])
print('there are',len(list_of_kernel_dicts_for_other_notebooks),'other kernels')
prev_kernel_count = len(list_of_kernel_dicts_for_other_notebooks)

start_time = time.time()
for indx in range(monitoring_duration_in_seconds_other_notebooks):
    row = {}
    system_row = row_for_system_performance(row)
    # detect new notebook kernels
    list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks)
    if len(list_of_kernel_dicts_for_other_notebooks)!=prev_kernel_count:
        print('\nkernel count changed from',prev_kernel_count)
        prev_kernel_count=len(list_of_kernel_dicts_for_other_notebooks)
        print('to',prev_kernel_count)
        print('elapsed time:',time.time()-start_time,'seconds')
        print(list_of_kernel_dicts_for_other_notebooks)
    
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        try:
            nb_dict[other_nb['pid']].append(row)
        except KeyError: # new pid for new kernel
            nb_dict[other_nb['pid']] = []
            nb_dict[other_nb['pid']].append(row)
    time.sleep(polling_period_in_seconds)

    #print('elapsed time:',time.time()-start_time,'seconds')
print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

there are 3 other kernels
elapsed time: 650.2085609436035 seconds


In [58]:
nb_df.keys()

dict_keys([44880, 44885, 44923])

In [59]:
for indx in range(len(list_of_kernel_dicts_for_other_notebooks)):
    nb_pid = list_of_kernel_dicts_for_other_notebooks[indx]['pid']
    list_of_kernel_dicts_for_other_notebooks[indx]['df']=nb_df[nb_pid]

In [60]:
with open("list_of_df_other_notebooks_"+str(time.time())+".pkl", 'wb') as handle:
    pickle.dump(list_of_kernel_dicts_for_other_notebooks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#   list_of_kernel_dicts_for_other_notebooks = pickle.load(handle)